In [1]:
import pandas as pd
import os
import re

In [2]:
def file_to_list(file_path):
    return (
        pd.read_csv(file_path, skiprows=2, header=None, encoding="utf-8")
        .iloc[:, 0]
        .tolist()
    )


def stardart_file_name_int(name):
    name = name.lower()
    n = int(re.search(r"\d+", name).group())
    typ = "stand" if "стоя" in name else "lying"
    ext = "rrn" if "rrn" in name else "rrg"
    return n, typ, ext


def standard_file_name_str(name):
    name = name.lower()
    n = str(re.search(r"\d+", name).group())
    typ = "stand" if "стоя" in name else "lying"
    ext = "rrn" if "rrn" in name else "rrg"
    return n, typ, ext


def folder_to_dataframe_int(dir_path):
    files = os.listdir(dir_path)
    nones = [None] * len(files)
    data = pd.DataFrame(
        {
            "n": nones,
            "stand_rrg": nones,
            "lying_rrg": nones,
            "stand_rrn": nones,
            "lying_rrn": nones,
        }
    )
    for count_lines, file in enumerate(files):
        n, typ, ext = stardart_file_name_int(file)
        file_path = os.path.join(dir_path, file)
        if os.path.isfile(file_path):
            if (data.n == n).any():
                if data.loc[data["n"] == n, typ + "_" + ext].isna().all():
                    data.at[data.index[data["n"] == n].tolist()[0], typ + "_" + ext] = (
                        file_to_list(file_path)
                    )
                else:
                    data.at[data.index[data["n"] == n].tolist()[0], typ + "_" + ext] = (
                        data.at[data.index[data["n"] == n].tolist()[0], typ + "_" + ext]
                        + file_to_list(file_path)
                    )
            else:
                data.at[count_lines, typ + "_" + ext] = file_to_list(file_path)
                data.at[count_lines, "n"] = n
    return data.dropna(how="all")


def folder_to_dataframe_str(dir_path):
    files = os.listdir(dir_path)
    nones = [None] * len(files)
    data = pd.DataFrame(
        {
            "n": nones,
            "stand_rrg": nones,
            "lying_rrg": nones,
            "stand_rrn": nones,
            "lying_rrn": nones,
        }
    )
    for count_lines, file in enumerate(files):
        n, typ, ext = standard_file_name_str(file)
        file_path = os.path.join(dir_path, file)
        if os.path.isfile(file_path):
            if (data["n"] == n).any():
                row_index = data.index[data["n"] == n].tolist()[0]
                if data.at[row_index, typ + "_" + ext] is None:
                    data.at[row_index, typ + "_" + ext] = file_to_list(file_path)
                else:
                    data.at[row_index, typ + "_" + ext] += file_to_list(file_path)
            else:
                data.at[count_lines, "n"] = n
                data.at[count_lines, typ + "_" + ext] = file_to_list(file_path)
    return data.dropna(how="all")

In [3]:
ah_data = folder_to_dataframe_str("../data/arterial_hypertension")
h_data = folder_to_dataframe_str("../data/healthy")
mi_data = folder_to_dataframe_str("../data/myocardial_infarction")
mi6_data = folder_to_dataframe_str("../data/myocardial_infarction_6mo")

In [4]:
ah_data.to_csv('arterial_hypertension.csv')
h_data.to_csv('healthy.csv')
mi_data.to_csv('myocardial_infarction.csv')
mi6_data.to_csv('myocardial_infarction_6mo.csv')